**DECISION TREE**

Importing libraries

In [101]:
import torch
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from dataset import TimeSeriesDataset
from dataset import NumpyToTensor
import sklearn
from sklearn.tree import DecisionTreeRegressor
import math

Importing data

In [102]:
train_dataa = pd.read_csv("data/train_delhi_aqi.csv")
test_dataa = pd.read_csv("data/test_delhi_aqi.csv")

In [103]:
# need to be tuned:
sequence_size = 100
num_levels = 6

Setting up dataloader

In [104]:
train_data = TimeSeriesDataset(train_dataa,sequence_size)
train_dataloader = DataLoader(dataset=train_data, batch_size = 100, shuffle=False, num_workers=2)

Converting the dataloader into an iterable

In [105]:
dataiter = iter(train_dataloader)

In [106]:
data = next(dataiter)

Creating the tree classes

In [107]:
# # Fit regression model
# regr_1 = DecisionTreeRegressor(max_depth=2)
# regr_2 = DecisionTreeRegressor(max_depth=5)
# regr_1.fit(X, y)
# regr_2.fit(X, y)

# # Predict
# X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
# y_1 = regr_1.predict(X_test)
# y_2 = regr_2.predict(X_test)


In [108]:
tree = DecisionTreeRegressor(max_depth=num_levels, criterion='squared_error', min_samples_split=0.0001)

Training Loop

In [110]:
epochs = 1
size = len(train_data)
niters = math.ceil(size/100)
print(size, niters)

2516454 25165


In [111]:
for ep in range(epochs):
    for i, (x, y) in enumerate(train_dataloader):
        tree.fit(x,y)
        if i%1000==0:
            ypred = tree.predict(x)
            err = torch.sum((y-ypred)**2)/len(y)
            print(f'Epoch {ep}, Iteration {i} train error: {err}')
    print(f'epoch {ep}/{epochs} completed')

Epoch 0, Iteration 0 train error: 0.5879152153808067
Epoch 0, Iteration 1000 train error: 3.3119931161559246
Epoch 0, Iteration 2000 train error: 0.9342633569107665
Epoch 0, Iteration 3000 train error: 1.9097843712792173
Epoch 0, Iteration 4000 train error: 0.7227043746905114
Epoch 0, Iteration 5000 train error: 1.3635483338539975
Epoch 0, Iteration 6000 train error: 0.8766080252613748
Epoch 0, Iteration 7000 train error: 0.9484942906829836
Epoch 0, Iteration 8000 train error: 1.3544544200213529
Epoch 0, Iteration 9000 train error: 1.194826685605162
Epoch 0, Iteration 10000 train error: 1.7871975371982374
Epoch 0, Iteration 11000 train error: 1.9800123553611655
Epoch 0, Iteration 12000 train error: 1.9137478698805501
Epoch 0, Iteration 13000 train error: 1.1180554799870672
Epoch 0, Iteration 14000 train error: 2.0426704356233376
Epoch 0, Iteration 15000 train error: 0.9102611846842124
Epoch 0, Iteration 16000 train error: 1.9001645992621585
Epoch 0, Iteration 17000 train error: 1.09179

In [112]:
test_data = TimeSeriesDataset(test_dataa,sequence_size)
test_dataloader = DataLoader(dataset=test_data, batch_size = len(test_data), shuffle=False, num_workers = 3)

In [113]:
#calc error
for ep in range(1):
    for i, (x,y) in enumerate(test_dataloader):
        ypred = tree.predict(x)
        error = torch.sum((y-ypred)**2)/len(y)
        print(f'iteratation {i}\n')

iteratation 0



In [114]:
error

tensor(2989.3518, dtype=torch.float64)

In [115]:
train_dataa

,PM2_5,PM10,NO2,SO2,CO_ug,Ozone,Temp,RH,WS,BP,WD,AQI (PM2.5)
31125,239.00,325.75,59.83,8.65,0.88,58.50,23.098722,78.08,1.30,988.10,346.37,288.600000
36812,35.77,50.43,28.75,4.15,0.63,23.37,23.098722,62.50,0.50,982.00,84.00,100.925001
56176,20.56,45.89,5.00,15.50,0.88,12.00,23.970000,76.75,0.68,1041.00,320.00,68.290597
66679,31.00,67.50,5.98,1.00,0.88,4.00,28.700000,97.97,0.34,991.52,228.25,90.598291
29992,80.63,350.94,70.74,0.90,0.88,17.35,18.590000,95.67,0.03,980.00,9.50,163.278946
...,...,...,...,...,...,...,...,...,...,...,...,...
44678,179.29,328.81,18.64,0.90,0.88,17.81,18.590000,9.50,0.47,1041.00,0.50,228.889993
33693,66.16,139.49,13.78,10.15,0.88,22.48,9.050000,75.10,0.49,976.55,85.83,155.663160
60993,502.25,67.50,31.67,1.00,0.88,4.00,22.830000,57.08,0.35,991.52,241.25,500.000000
102883,39.87,243.61,16.51,21.72,0.88,47.20,18.150000,32.00,2.54,1008.80,234.60,111.174997
